In [1]:
import pandas as pd #import pandas library to handle file, database

In [2]:
dataset = pd.read_csv("insurance_pre.csv") #read the file data and store it in dataset variable

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
#data preprocessing - because categorial data can't be handle so we modify the data into numerical data using one hot encoding(Nominal)

dataset = pd.get_dummies(dataset, drop_first=True)

dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [9]:
boolean_columns = dataset.select_dtypes(include=['bool']).columns #get boolean type column data from the dataset and store it in boolean_columns variable
print(boolean_columns)

dataset[boolean_columns] = dataset[boolean_columns].astype(int) #convert boolean column dataset to 0's and 1's
print(dataset[boolean_columns])

Index(['sex_male', 'smoker_yes'], dtype='object')
      sex_male  smoker_yes
0            0           1
1            1           0
2            1           0
3            1           0
4            1           0
...        ...         ...
1333         1           0
1334         0           0
1335         0           0
1336         0           0
1337         0           1

[1338 rows x 2 columns]


In [11]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [13]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [15]:
#split input data
independent = dataset [['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [17]:
dependent = dataset[['charges']] #split output data

dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor #Create a model
from sklearn.tree import DecisionTreeRegressor

base = DecisionTreeRegressor(max_depth=4)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.5, 0.8],
    'loss': ['linear', 'square', 'exponential'],
    'random_state': [0]}

grid = GridSearchCV(AdaBoostRegressor(estimator=base), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='r2')

grid.fit(independent, dependent)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=4)),
             n_jobs=-1,
             param_grid={'learning_rate': [0.5, 0.8],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [100, 200, 300], 'random_state': [0]},
             scoring='r2', verbose=3)

In [43]:
re = grid.cv_results_

#print("CV_RESULTS : ", re)

grid_pred = grid.predict(independent) #test the model with input data

from sklearn.metrics import r2_score #Evaluation metrics

r_score = r2_score(dependent, grid_pred)
print("R score value is : ", r_score) 

print("Best Score:", grid.best_score_)
print("Best Parameters:", grid.best_params_)

R score value is :  0.8358527637329298
Best Score: 0.7732786320853242
Best Parameters: {'learning_rate': 0.5, 'loss': 'linear', 'n_estimators': 100, 'random_state': 0}


In [45]:
import pandas as pd

# feature names 
feature_names = ['age', 'bmi', 'children', 'sex_male', 'smoker_yes']  

independent_df = pd.DataFrame(independent, columns=feature_names)

# Get feature importances from best model inside GridSearchCV
importance_scores = grid.best_estimator_.feature_importances_

# Create DataFrame of feature importances
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance_scores
})

# Sort by most important feature
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the result
print(importance_df)


      Feature  Importance
4  smoker_yes    0.647183
1         bmi    0.198370
0         age    0.135170
2    children    0.018519
3    sex_male    0.000758


In [47]:
table = pd.DataFrame.from_dict(re) #convert dictonary format to DataFrame(table)

In [49]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_n_estimators,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.178896,0.053861,0.015967,0.002445,0.5,linear,100,0,"{'learning_rate': 0.5, 'loss': 'linear', 'n_es...",0.789250,0.729813,0.823723,0.764396,0.759211,0.773279,0.031514,1
1,0.187642,0.053655,0.022513,0.007027,0.5,linear,200,0,"{'learning_rate': 0.5, 'loss': 'linear', 'n_es...",0.789250,0.729813,0.823723,0.764396,0.759211,0.773279,0.031514,1
2,0.185476,0.037824,0.015387,0.003011,0.5,linear,300,0,"{'learning_rate': 0.5, 'loss': 'linear', 'n_es...",0.789250,0.729813,0.823723,0.764396,0.759211,0.773279,0.031514,1
3,0.512011,0.021443,0.036928,0.007194,0.5,square,100,0,"{'learning_rate': 0.5, 'loss': 'square', 'n_es...",0.622678,0.532811,0.569103,0.585420,0.575891,0.577180,0.028891,10
4,1.010362,0.072020,0.067778,0.003337,0.5,square,200,0,"{'learning_rate': 0.5, 'loss': 'square', 'n_es...",0.608140,0.498745,0.543679,0.565663,0.545988,0.552443,0.035437,14
5,1.478149,0.071037,0.093788,0.004179,0.5,square,300,0,"{'learning_rate': 0.5, 'loss': 'square', 'n_es...",0.598165,0.489588,0.524254,0.557399,0.536272,0.541136,0.036007,16
6,0.545779,0.033733,0.037600,0.006355,0.5,exponential,100,0,"{'learning_rate': 0.5, 'loss': 'exponential', ...",0.655521,0.569701,0.623809,0.638568,0.620151,0.621550,0.028772,7
7,1.001956,0.027946,0.064781,0.005561,0.5,exponential,200,0,"{'learning_rate': 0.5, 'loss': 'exponential', ...",0.623800,0.522588,0.567491,0.596916,0.578169,0.577793,0.033574,9
8,1.467314,0.026383,0.092358,0.006382,0.5,exponential,300,0,"{'learning_rate': 0.5, 'loss': 'exponential', ...",0.608361,0.504904,0.549215,0.580549,0.555654,0.559737,0.034443,11
9,0.199269,0.151988,0.016189,0.008265,0.8,linear,100,0,"{'learning_rate': 0.8, 'loss': 'linear', 'n_es...",0.631869,0.677234,0.724948,0.792607,0.858074,0.736946,0.080648,4


In [51]:
age = int(input("Enter the age:"))
bmi = float(input("Enter the BMI:"))
children = int(input("Enter the children number:"))
sex = int(input("Enter sex (0 for female and 1 for male):"))
smoker = int(input("Enter 0 for non smoker and 1 for smoker:"))

Enter the age: 26
Enter the BMI: 20
Enter the children number: 0
Enter sex (0 for female and 1 for male): 0
Enter 0 for non smoker and 1 for smoker: 1


In [53]:
Final_prediction =grid.predict([[age,bmi,children,sex,smoker]])
print("Final_prediction of Adaboost: {}".format(Final_prediction))

Final_prediction of Adaboost: [18232.10950067]


C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(


In [55]:
#save the model using pickle library

import pickle
fileName = "Adaboost_grid.sav"
pickle.dump(grid, open(fileName, "wb")) #saved model available in regressor variable. Now storing that variable into fileName

In [57]:
loaded_model = pickle.load(open("Adaboost_grid.sav","rb"))
result = loaded_model.predict([[30,30,2,1,1]])  #30,25.899,2,1,1

C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(


In [59]:
result

array([23036.3093075])